# Nuestras Caras

Este código implementa una técnica de reconocimiento facial conocida como "Eigenfaces". 

La idea básica detrás de Eigenfaces es representar las caras de un conjunto de datos de entrenamiento en un espacio de características de dimensionalidad reducida utilizando el análisis de componentes principales (PCA). 

Luego, las nuevas caras se proyectan en este espacio de características y se comparan con las caras de entrenamiento para reconocerlas.

DEPENDENCIAS NECESARIAS:

Previo a ejecutar la notebook deben ser instaladas las siguientes dependencias:
* pip install opencv-python
* pip install pandas
* pip install matplotlib
* pip install scikit-learn

In [ ]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.cm as cm
from sklearn.decomposition import PCA
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split
from photo_30x30 import cortar_imagenes     # Función para recortar imagenes y pasar a grises
from pixels import intensidad_pixels    # Función para obtener la intensidad de los pixeles
import perceptrones as per        # Script con backpropagation

In [ ]:
#********************************************************************************************************************
#                    CORTE DE LAS FOTOS Y CAMBIO DE ESCALA DE GRISES MEDIANTE PHOTO_30X30.PY
#********************************************************************************************************************
# Definimos donde se encuentran las fotos crudas y donde iran las grises 30x30
#current_directory = os.getcwd()
folder_name_raw = "input"
folder_name_output = "output"

# current_directory
# os.chdir(os.path.join(current_directory, "NuestrasCaras"))


# Cortar las fotos y cambiar a escala de grises
# cortar_imagenes(folder_name_raw, folder_name_output)


In [ ]:
#********************************************************************************************************************
#                    OBTENCION DE LOS VALORES DE LOS PIXELES DE LAS FOTOS MEDIANTE PIXELS.PY
#********************************************************************************************************************
# Guardo la info de las fotos en un dataframe
folder_path_output = os.path.join(os.getcwd(), folder_name_output)
data_fotos = intensidad_pixels(folder_path_output)

# Get the file names
file_names = data_fotos.iloc[:, 0]

# Tomo los nombres de cada persona
people_names = [name.split("-")[0] for name in file_names]
# Transformo los nombres a números (0-1) usando label binarizer
nombres = LabelBinarizer().fit_transform(people_names)
print(people_names)
print(nombres)

# Get the greyscale values
greyscale_values = data_fotos.iloc[:, 1:].values/255.0

#********************************************************************************************************************
# Separamos los data sets en training y test
#********************************************************************************************************************
X_train, X_test, y_train, y_test = train_test_split(greyscale_values, nombres, test_size=0.2, random_state=42)

#********************************************************************************************************************

In [ ]:
# Dimensiones del array
people_names = np.array(people_names)
people_names.shape

greyscale_values = np.array(greyscale_values)
greyscale_values.shape


In [ ]:
# obtiene y mustra la cara promedio
cara_promedio = np.mean(greyscale_values, axis=0)
cara_promedio.shape
plt.figure(figsize=(3, 2))
plt.imshow(cara_promedio.reshape(30,30),cmap="gray")
plt.title('Cara Promedio')

In [ ]:
#********************************************************************************************************************
#                    PCA
#********************************************************************************************************************
# Se aplica PCA a las imagenes
# IMPORTANTE! al aumentar el nro de componentes tambien aumenta la precision en la comparacion de imagenes para la clasificacion al final de la notebook!
# Tiene sentido porque aumento el porcentaje de variabilidad explicada por las componentes, acercandome mas a la realidad
n_pca = 60
pca = PCA(n_components=n_pca).fit(greyscale_values)

eigenfaces = pca.components_
varianza_explicada = pca.explained_variance_ratio_

# plot explained variance (grafico continuo)
plt.figure(figsize=(8, 4))
plt.plot(np.cumsum(varianza_explicada))
plt.xlabel('Number of components')
plt.ylabel('Cumulative explained variance')
plt.title('Explained variance vs number of components')
plt.show()


Se observa que con aproximadamente 40 CP explicamos el 80% de la varianza. Y con 60 CP, nos acercamos al 90%.

In [ ]:
#********************************************************************************************************************
#                    GRAFICO DE LAS PRIMERAS 15 EIGENFACES
#********************************************************************************************************************

fig, ax = plt.subplots(4, 4, figsize=(7, 7))

ax[0, 0].imshow(cara_promedio.reshape(30,30), cmap="gray")
ax[0, 0].set_title("Cara promedio")

for i in range(0,15): # primeras 15 pca
    ax[(i+1) // 4, (i+1) % 4].imshow(eigenfaces[i].reshape(30,30), cmap="gray")
    ax[(i+1) // 4, (i+1) % 4].set_title(f"Eigenface {i+1}")

#plot dimensions
plt.tight_layout()
plt.show()

### Entrenamiento de la red neuronal con backpropagation
Comienza inicializando las matrices de pesos de manera aleatoria y realizando la propagación hacia adelante para obtener las salidas de la red. Luego, se calcula el error promedio y se inicia un bucle de entrenamiento que continúa hasta que se cumpla un criterio de convergencia o se alcance el límite de épocas. En cada iteración, se realiza la propagación hacia adelante y hacia atrás para ajustar los pesos mediante la corrección de errores. El proceso se repite hasta que se alcance la convergencia o el límite de épocas, mostrando el número de época y el valor del error en cada iteración.

In [ ]:
#********************************************************************************************************************
# AHORA TENEMOS QUE USAR EL BACKPROPAGATION 
#********************************************************************************************************************

# Tomo los valores del data set de entrenamiento
X = pca.fit_transform(X_train)

# Tomo los nombres de las personas del data set de entrenamiento
Y = y_train

# PRIMERA PRUEBA
# Entrenamiento de la red neuronal utilizando el perceptron backpropagation
# Aleatoriamente tomo esta cantidad de neuronas en la primera y segunda capa
n_neuronas_capa_1 = 19
n_neuronas_capa_2 = 7
red_neuronal = per.backpropagation_2_capas(X, Y, n_neuronas_capa_1, n_neuronas_capa_2, 2000, 1.0e-6, 0.3)

# Tomo los pesos de la red entrenada
pesos = red_neuronal[:6]


In [ ]:
# Predecimos cara

# Tomo una foto de prueba
import random
n = random.randint(0, len(greyscale_values))
foto_prueba = greyscale_values[n]
print(nombres[n])
print(people_names[n])
foto_prueba_pca = pca.transform(foto_prueba.reshape(1, -1))

# Prediccion
prediccion = per.predecir_clase_2_capas(pesos, foto_prueba_pca)

print(prediccion[-1])   # La prediccion arroja varios valores, el mas alto es el correspondiente a la que tiene mas probabilidad de ser
print(people_names[prediccion[-1]])  # La persona que es


In [ ]:
# Evaluamos la mejor combinacion de neuronas en las capas ocultas 1 y 2 (DEMORA MUCHO!)

# Creo una lista vacia para ir guardando los resultados
resultados = []

# Creo tres loops: 
# uno para la cantidad de neuronas en la capa 1, 
# otro para la cantidad de neuronas en la capa 2,
# y otro para la cantidad de fotos del test set

import time

for i in range(1,36):
    for j in range(1,36):
        start_time = time.time()
        red_neuronal = per.backpropagation_2_capas(X, Y, i, j, 2000, 1.0e-6, 0.3)
        pesos = red_neuronal[:6]
        aciertos = 0
        # tiempo de ejecucion
        tiempo = time.time() - start_time
        # epoch y error
        epoch = red_neuronal[-1]
        error = red_neuronal[-2]
        # Recorro las fotos del test set
        for k in range(0, len(y_test)):
            foto_prueba = X_test[k]
            foto_prueba_pca = pca.transform(foto_prueba.reshape(1, -1))
            prediccion = per.predecir_clase_2_capas(pesos, foto_prueba_pca)
            # Si la prediccion es correcta, suma 1 en aciertos
            if prediccion[-1] == np.where(y_test[k]==1)[0][0]:
                aciertos += 1
        # Imprime combinacion de neuronas y aciertos
        print(f"*** Neuronas en primera capa: {i} *** Neuronas en segunda capa: {j} >>> Aciertos: {aciertos} ({round(aciertos/len(y_test)*100,2)}%)")
        print('')
        # Guarda los resultados de esta combinacion en la lista
        resultados.append((i, j, tiempo, epoch, error, aciertos))



#********************************************************************************************************************
# Tarda tanto en correr que guardo los resultados en un archivo externo

# Guardo los resultados en un dataframe
resultados_df = pd.DataFrame(resultados, columns=["Neuronas 1", "Neuronas 2", "Tiempo", "Epoch", "Error", "Aciertos"])

# Guardo el dataframe en un archivo csv
resultados_df.to_csv("test_backpropagation.csv", index=False)


In [ ]:
# Graficamos los resultados de la evaluacion de las neuronas en las capas ocultas

resultados = np.array(resultados)

# Paso los aciertos a porcentaje
resultados_pct = resultados[:,2]

# Ploteo en un grafico 2D
plt.figure(figsize=(10, 6))
plt.scatter(resultados[:,0], resultados[:,1], c=resultados_pct, cmap='viridis', s=100)
plt.colorbar()
plt.xlabel('Neuronas en capa 1')
plt.ylabel('Neuronas en capa 2')
plt.title('Aciertos en funcion de las neuronas en las capas ocultas')
plt.show()

# Ploteo en un grafico 3D
fig = plt.figure(figsize=(10, 6))
ax = fig.add_subplot(111, projection='3d')
ax.scatter(resultados[:,0], resultados[:,1], resultados_pct, c=resultados_pct, cmap='viridis')
ax.set_xlabel('Neuronas en capa 1')
ax.set_ylabel('Neuronas en capa 2')
ax.set_zlabel('Aciertos (%)')
plt.title('Aciertos en funcion de las neuronas en las capas ocultas')
plt.show()

#********************************************************************************************************************





In [ ]:
# Armo un ranking de las 15 mejores combinaciones segun el porcentaje de aciertos
resultados[:,2] = resultados_pct
ranking = resultados[np.argsort(resultados_pct)[::-1]]
print(ranking[:15])

# Nos quedamos con la mejor combinacion de neuronas en las capas ocultas
# (19, 19) es la mejor combinacion
# Pero (12, 8) es casi igual y tiene mucha menor cantidad
